# 1. Resumo

   A **Divvy Trips** é uma empresa de compartilhamento de bicicletas na cidade de Chicago. A empresa apresenta dois tipos de planos, o casual (passageiros casuais) e o anual (membros). Os analistas financeiros concluíram que os membros anuais são mais lucrativos do que os passageiros casuais e o departamento de marketing acredita que a chave para o crescimento da empresa seria maximizar o número de membros, convertendo ciclistas casuais em anuais, ao invés de criar campanhas para novos clientes.
   
   O objetivo desta análise é criar estratégias de marketing destinadas a converter os passageiros casuais em membros. Para isso, é preciso entender melhor como ambos os tipos de clientes utilizam o serviço, por que os passageiros casuais iriam querer adquirir um plano anual e qual estratégia de marketing seria adotada. Serão analisados os históricos de trajetos de bicicleta para identificar as tendências.
   
   Para isso, inicialmente são feitas as seguintes perguntas:
   * Como os membros anuais e os ciclistas casuais utilizam as bicicletas?
   * Por que os passageiros casuais iriam querer adquirir planos anuais?
   * Qual a estratégia de marketing poderá ser adotada para influenciar os passageiros casuais a se tornarem membros?
    
   Essas perguntas têm o objetivo de entender os dados, além de responder o problema em questão: vale a pena investir na conversão de passageiros casuais em membros? Se sim, qual a estratégia a ser adotada?
   
 > Além desse relatório, foi desenvolvido um dashboard, utilizando o Power BI, para reforçar os insights encontrados. O dashboard pode ser acessado através desse [link](https://app.powerbi.com/view?r=eyJrIjoiZjYxMzlkMDQtOTgxYS00MDY1LTgwMDctYmE3ZTk4MTg5NzM1IiwidCI6IjAyMWExOTNmLTVjN2UtNGMyZS1hZjllLWU3MTMxN2NlYmViMyJ9).

# 2. Preparação dos dados

São utilizados dados históricos de trajetos para analisar e identificar as tendências. será utilizado um conjunto de dados públicos, disponibilizados pela Motivate International Inc. sob esta [licença](https://ride.divvybikes.com/data-license-agreement) (**Atualmente, esse link está fora do ar, por motivos desconhecidos**). sendo, portanto, dados válidos para o propósito do estudo de caso. Os dados são disponibilizados através deste [link](https://divvy-tripdata.s3.amazonaws.com/index.html).

Será feita uma análise anual, iniciando em julho de 2021 até junho de 2022. Os dados estão disponibilizados de forma mensal, portanto devem ser mesclados para o estudo do período indicado.

## 2.1. Dicionário de Dados

**ride_id:** Identificação de cada passeio

**rideable_type:** Tipo de bicicleta utilizada

**started_at:** Data e hora do início do passeio

**ended_at:** Data e hora do fim do passeio

**start_station_name:** local de início do passeio (onde a bicicleta foi alugada)

**start_station_id:** Identificação do local de início do passeio

**end_station_name:** local do fim do passeio (onde a bicicleta foi entregue)

**end_station_id:** Identificação do local do fim do passeio

**start_lat:** latitude do local de início do passeio

**start_lng:** longitude do local de início do passeio

**end_lat:** latitude do local de fim do passeio

**end_lng:** longitude do local de fim do passeio

**member_casual:** tipo do cliente (membro ou casual)


# 3. Processamento dos dados

## 3.1 - Ajuste Inicial do Conjunto de Dados

Algumas manipulações serão realizadas no conjunto de dados, com o intuito de prepará-lo para futuras análises. Como foi mencionado anteriormente, os dados brutos estão separados por mês, portanto o primeiro passo será a mescla desses arquivos (12 no total). Os arquivos de cada mês foram baixados e extraídos para uma pasta específica na máquina.

In [1]:
# Importanção das bibliotecas utilizadas no projeto

# Pandas: biblioteca de analise de dados e ferramentas de manipulação de conjunto de dados.
# Numpy: biblioteca para mainpulação de arrays
import pandas as pd
import numpy as np

# OS: biblioteca utilizada para interação entre Python e Sistema Operacional.
import os

# Bibliotecas gráficas
import matplotlib.pyplot as plt
import seaborn as sns

# Biblioteca utilizada para suprimir possíveis warnings (avisos) que aparecem ao rodar o código. 
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Listando os arquivos baixados

# caminho da pasta dos arquivos mensais
caminho = r'C:\Users\luisf\Desktop\database\dados_brutos'
diretorio = os.listdir(caminho)

diretorio

In [ ]:
# Analisando a quantidade de colunas de cada arquivo csv.
for arquivo in diretorio:
    caminho_arquivo = os.path.join(caminho, arquivo)
    df = pd.read_csv(caminho_arquivo)
    print(df.shape[1])

> Todos os arquivos possuem 13 colunas. Ao observá-los individualmente (abrindo em um editor de texto, por exemplo), vemos que todas as colunas são iguais, portanto é possível realizar a mescla desses arquivos sem a necessidade de nenhuma modificação.

In [ ]:
# coletando o caminho de cada coluna.

url_arquivos = []
for arquivo in diretorio:
    caminho_arquivo = os.path.join(caminho, arquivo)
    url_arquivos.append(caminho_arquivo)

url_arquivos

In [ ]:
%%time
# mescla dos arquivos em um dataframe, utilizando os caminhos de cada arquivo
df_anual = pd.concat(map(pd.read_csv, url_arquivos), ignore_index=True)

In [ ]:
df_anual.shape

In [ ]:
df_anual.head()

In [ ]:
df_anual.info()

In [ ]:
# Excluindo algumas colunas irrelevantes para a análise (colunas de ID)
df_anual.drop(['ride_id', 'start_station_id', 'end_station_id'], inplace=True, axis=1)

In [ ]:
# Excluindo as colunas de latitude e longitude, pois não serão utilizadas
df_anual.drop(['start_lat', 'start_lng', 'end_lat', 'end_lng'], inplace=True, axis=1)

> Existe uma pequena variação das latitudes e longitudes de uma mesma estação no conjunto de dados. É algo natural, já que esses dados, provavelmente, são coletados por meio de um aparelho geolocalizador e a simples diferença de localidade entre pontos de retirada e entrega das bicicletas, em uma mesma estação, podem gerar essas pequenas mudanças. Por isso, não utilizaremos essas informações.

> Essas variações podem ser vistas, por exemplo, na tabela retornada anteriormente, em que a estação `California Ave & Cortez St`apresenta 2 latitudes diferentes. O mesmo vale para a longitude.

In [ ]:
# Modificação dos nomes das colunas - Traduzindo os nomes das colunas
df_anual.columns = ['Tipo Bicicleta',
                    'Início Aluguel',
                    'Fim Aluguel',
                    'Estação Inicial', 
                    'Estação Final', 
                    'Tipo Plano']

In [ ]:
df_anual.head()

## 3.2 Tratamento dos Tipos de dados

As colunas `Início Aluguel` e `Fim Aluguel` possuem informações de data, porém esses dados foram interpretados pelo Python como tipo object (string). Será realizada a conversão para o tipo datetime (data e hora).

In [ ]:
df_anual.info()

In [ ]:
df_anual['Início Aluguel'] = pd.to_datetime(df_anual['Início Aluguel'], errors='coerce')
df_anual['Fim Aluguel'] = pd.to_datetime(df_anual['Fim Aluguel'], errors='coerce')

In [ ]:
df_anual.info()

## 3.2 Valores nulos

As colunas `Estação Inicial` e `Estação Final` apresentam, respectivamente 14.17% e 15.12% de valores nulos, em relação ao total. Uma ideia inicial para o tratamento desses dados seria utilizar as latitudes e longitudes para encontrar a localização desses registros. Portanto, como mencionado anteriormente, essas informações, para uma mesma estação, variam ligeiramente ao longo do conjunto de dados.

Analisaremos agora, qual o impacto gerado no conjunto de dados com a possível exclusão desses valores.

In [ ]:
round(((df_anual.isnull().sum()/df_anual.shape[0])*100), 2)

In [ ]:
dados_nulos_inicio = df_anual['Estação Inicial'].isnull()
dados_nulos_fim = df_anual['Estação Final'].isnull()

In [ ]:
# Filtrando todos os registros com dados nulos - utilizando o comando "OR" (|)
dados_nulos = df_anual[dados_nulos_inicio | dados_nulos_fim]
dados_nulos

In [ ]:
# Porcentagem de registros com dados nulos
round((dados_nulos.shape[0]/df_anual.shape[0])*100, 2)

In [ ]:
# Porcentagem da coluna Tipo Plano para os registros com dados nulos
round((dados_nulos['Tipo Plano'].value_counts()/dados_nulos.shape[0])*100, 2)

In [ ]:
# Porcentagem da coluna Tipo Plano para o conjutn ode dados total
round((df_anual['Tipo Plano'].value_counts()/df_anual.shape[0])*100, 2)

> Inicialmente, os registros com valores nulos (ou para estação inicial ou para estação final) foram filtrados. Temos, portanto, um total de 1.221.514 registros com dados faltantes, o que equivale à 20.7% do total. É uma alta porcentagem, porém como o conjunto de dados apresenta um grande volume de registros, a exclusão dessas linhas não tornaria a análise mais pobre.

> Após isso, calculou-se a porcentagem para cada categoria (membro ou casual) da variável `Tipo Plano`, tanto para o conjunto de dados faltantes, quanto para o conjunto total. Percebe-se a relação membro/casual é similar (55/45 aproximadamente) para os dois conjuntos de dados. Portanto, o impacto negativo causado pela exclusão desses registros com dados faltantes, para a coluna `Tipo Plano` é muito baixo, já que não afetaria na porcentagem das categorias dessa variável.

> A análsie foi realizada utilizando a variável `Tipo Plano` pois ela é o coração do problema de negócio. 

**Decisão:** exclusão dos registros que possuem dados faltantes.

In [ ]:
# Apagando as linhas com valores nulos
df_anual.dropna(inplace=True)

In [ ]:
df_anual.info()

In [ ]:
round(((df_anual.isnull().sum()/df_anual.shape[0])*100), 2)

## 3. 3 Manipulação das Colunas de Data

A partir das colunas de data, serão criadas algumas novas colunas, com o objetivo de analisar os dados considerando granularidades diferentes. Da coluna `Início Aluguel` obteremos as informações de mês e dia da semana. Além disso, calcularemos a duração do aluguel, fazendo a difeernça entre as colunas `Fim Aluguel` e `Início Aluguel`. 

In [ ]:
# Criando a coluna Mês
# A utilização de loc = 1 coloca a nova coluna no índice 1 do conjunto de dados.
df_anual.insert(loc = 1, column = 'Mês', value = df_anual['Início Aluguel'].dt.month)

df_anual.head()

In [ ]:
# Mudando a localização para pt_BR, para obter o dia da semana em português.

import locale
locale.setlocale(locale.LC_TIME, 'pt_BR')

In [ ]:
# Criando a coluna Dia da Semana
df_anual.insert(loc = 2, column = 'Dia da Semana',
                value = df_anual['Início Aluguel'].apply(lambda x : x.strftime('%A')))

In [ ]:
# Criando a coluna Duração
df_anual.insert(loc = 5, column = 'Duração',
                value = df_anual['Fim Aluguel'] - df_anual['Início Aluguel'])

In [ ]:
df_anual.head()

In [ ]:
df_anual.info()

In [ ]:
# Salvando os dados manipulados
df_anual.to_csv('divvy_tripdata_ckpoint1.csv', index_label = False)

# 4. Análise Exploratória de Dados

In [2]:
# Criando checkpoint 1.
df = pd.read_csv('divvy_tripdata_ckpoint1.csv')

In [ ]:
df.info()

## 4.1 Por Tipo de Bicicleta:

In [ ]:
round((df['Tipo Bicicleta'].value_counts()/df.shape[0])*100, 2)

> A grande maioria dos aluguéis são de biciletas clássicas (comuns), com 68%. Temos também bicicletas elétricas (26.6%) e por fim, as biciletas docked (mais de um lugar), representando 5% dos alugéis. 

In [ ]:
df_anual.groupby(['Tipo Plano', 'Tipo Bicicleta'])[['Início Aluguel']].count().sort_values('Tipo Plano', ascending=False)

> Apenas os clientes casuais alugam bicicletas docked. Tanto os membros quanto os casuais alugam mais bicicletas clássicas.

## 4.2 Por Mês e Dia da Semana

In [ ]:
print('Porcentagem dos alugueis por mês')
print(round((df['Mês'].value_counts()/df.shape[0])*100, 2))
print('\n' + 50 * '-'+ '\n')
print('Porcentagem dos alugueis por dia da semana')
print(round((df['Dia da Semana'].value_counts()/df.shape[0])*100, 2))

> Existe uma crescente de alugueis do mês 5 (maio) ao mês 10 (outubro), sendo os meses de julho e agosto com mais registros.

> A proporção de registros por dia da semana é bem equilibrada, com um ligeiro aumento no sábado.

In [ ]:
plt.figure(figsize = (10, 8))
chart = sns.countplot(data = df,
                     x = 'Mês',
                     color = '#279259',
                     alpha = 0.9)

# Labels
sns.set(font_scale = 1.5)
sns.set_style('whitegrid')
chart.set_xlabel('\nMês', fontsize = 14)
chart.set_ylabel(' ')

plt.title('Quantidade de Aluguéis por Mês', size = 20);

In [ ]:
ordem = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']

plt.figure(figsize = (10, 8))
chart = sns.countplot(data = df,
                     x = 'Dia da Semana',
                     color = '#0058A9',
                     alpha = 0.9,
                     order = ordem)

# Labels
sns.set(font_scale = 1.5)
sns.set_style('whitegrid')
chart.set_xlabel('\nDia da Semana', fontsize = 14)
chart.set_ylabel(' ')

plt.title('Quantidade de Aluguéis por Dia da Semana', size = 20)
plt.xticks(rotation = 45);

## 4.3 Por Duração do Aluguel

A coluna `Duração`, ao ser criada, possuía o tipo timedelta (diferença de tempo). Porém, ao recarregar o conjunto de dados, essa coluna foi interpretada pelo Python como object. 

In [4]:
df['Duração'] = pd.to_timedelta(arg = df['Duração'])

In [5]:
# Convertendo a duração do aluguel para minutos, para padronizar os dados
df['Duração_Minutos'] = (df['Duração'].dt.total_seconds()/60).round(2)

In [6]:
df['Duração_Minutos'].describe()

count    4.678871e+06
mean     1.926107e+01
std      1.186626e+02
min     -1.290800e+02
25%      6.520000e+00
50%      1.142000e+01
75%      2.057000e+01
max      4.910715e+04
Name: Duração_Minutos, dtype: float64

> Da tabela acima, temos várias informações estatísticas sobre a coluna `Duração_Minutos`, como, por exemplo, a média, os quartis, máximo e mínimo. Algumas observações:

* Valor mínimo negativo: não faz sentido termos aluguéis com duração negativa, sendo considerado, portanto, um erro de coleta de dados.

* O valor máximo da duração, é cerca de 49 mil minutos, o que representa mais de 34 dias de duração. Claramente temos alguns outliers, que serão analisados.

In [ ]:
# Quantidade de registros negativos
df[df['Duração_Minutos'] < 0].shape[0]

In [7]:
# Excluindo os registros com duração dos aluguéis negativa
df = df[~(df['Duração_Minutos'] < 0)]

In [ ]:
df['Duração_Minutos'].describe()

> Agora temos como valores mínimos 0, o que representaria uma duração de 0 minutos.

In [ ]:
plt.figure(figsize = (15, 6))

plt.subplot(1, 2, 1)
sns.boxplot(df['Duração_Minutos'], color = '#FF4040')
plt.title('Boxplot Duração dos Aluguéis', size = 18);

plt.subplot(1, 2, 2)
df['Duração_Minutos'].hist(color = 'green', alpha = 0.7, edgecolor='black')
plt.title('Histograma Duração dos Aluguéis', size = 18);

plt.tight_layout()

> Temos diversos outliers, representado por durações de aluguéis muito diferentes da média de 19 minutos (informação retirada do describe acima). A partir dessa análise inicial, algumas inferências serão feitas:

1. Durações de aluguéis abaixo de 5 minutos serão consideradas desistências ou erros de coletas de dados;
2. Aluguéis com mais de 6 horas (360 minutos) também não serão considerados.

**obs:** em um caso real, essas decisões deveriam ser tomadas em conjunto com a área de negócios, já que haveria mais informação sobre os dados. 

In [8]:
df = df[(df['Duração_Minutos'] < 6 * 60) & (df['Duração_Minutos'] > 5)]

In [ ]:
df['Duração_Minutos'].describe()

In [ ]:
plt.figure(figsize = (15, 6))

plt.subplot(1, 2, 1)
sns.boxplot(df['Duração_Minutos'], color = '#FF4040')
plt.title('Boxplot Duração dos Aluguéis', size = 18);

plt.subplot(1, 2, 2)
df['Duração_Minutos'].hist(color = 'green', alpha = 0.7, edgecolor='black')
plt.title('Histograma Duração dos Aluguéis', size = 18);

plt.tight_layout()

> Como podemos perceber, ainda existem outliers, porém são valores que acredito que realmente podem existir.

## 4.4 Por Estação (Início e Final)

In [ ]:
print('Estações Iniciais\n')
print(df['Estação Inicial'].value_counts())
print('\n' + 50 * '-'+ '\n')
print('Estações Finais\n')
print(df['Estação Final'].value_counts())

> As 5 primeiras estações são as mesmas para início e fim do aluguel. São elas, portanto, que apresetam maiores movimentações de clientes.

In [10]:
inicial = df["Estação Inicial"].value_counts()
final = df["Estação Final"].value_counts()

In [11]:
dicionario = {'Qtd Início': inicial, 'Qtd Fim': final}

df_qtd_estacao = pd.concat(dicionario, axis = 1)

In [13]:
df_qtd_estacao['Total'] = df_qtd_estacao['Qtd Início'] + df_qtd_estacao['Qtd Fim']

In [14]:
df_qtd_estacao.head()

,Qtd Início,Qtd Fim,Total
Streeter Dr & Grand Ave,75629.0,77276.0,152905.0
Michigan Ave & Oak St,39921.0,40510.0,80431.0
DuSable Lake Shore Dr & North Blvd,36915.0,41167.0,78082.0
Millennium Park,35685.0,36781.0,72466.0
Wells St & Concord Ln,35556.0,35304.0,70860.0


> Acima temos as 5 estações com maiores movimentações de clientes.

## 4.5 Por Tipo de Plano

In [ ]:
round((df['Tipo Plano'].value_counts()/df.shape[0])*100, 2)

> Temos uma relação **53/47**, aproximadamente, para os clientes que são membros e os que são casuais.

## 4.6 Nova Variável

Uma suposição feita, após toda a análise exploratória é que clientes, tanto membros, quanto casuais, utilizam as biciletas para o deslocamento ao trabalho. Portanto, criaremos uma nova variável que tente representar se aquele registro é ou não para esse fim.

Visualizando o boxplot e o histograma, para a variável `Duração_Minutos` podemos perceber que a grande maioria dos aluguéis são de até 50 minutos. Acredito que isso pode ser considerado uma duração aceitável para o deslocamento ao trabalho.

Além disso, consideraremos que a maioria das pessoas não trabalham nos finais de semana. Portanto, a nova variável irá considerar essas duas possibilidades. Aluguéis de até 50 minutos de duração e que não sejam no fim de semana (observe a utilização do "e", indicando uma concatenação dessas duas situações).

> **obs:** vale destacar que essa nova variável indica uma suposição nos dados e não uma certeza. Ela poderá não englobar todos os registros de pessoas que vão ao trabalho (como, por exemplo, pessoas que usam a bicicleta aos fins de semana para o trabalho), além de englobar possíveis registros de pessoas que alugaram a bicicleta sem a intenção de trabalhar, mas que se encaixam nas condições.

In [ ]:
# Criando a nova variável Trabalho

# Inicialmente, criaremos uma nova coluna com valores nulos. Caso as condições impostas no próximo passo possua algum erro,
# ou ainda, não consiga englobar todos os dados, existirão valores nulos nessa nova coluna.
df['Trabalho'] = np.nan

# Criando a nova coluna passando as condições impostas
df['Trabalho'] = np.where(
    (df['Duração_Minutos'] < 50) & (df['Dia da Semana'] != 'sábado') & (df['Dia da Semana'] != 'domingo'), "Sim", "Não")  

In [ ]:
df['Trabalho'].isnull().sum()

In [ ]:
df.sample(10)

In [ ]:
round((df['Trabalho'].value_counts()/df.shape[0])*100, 2)

> A maior parte dos dados se encaixam nas condições especificadas (63% aproximadamente).

In [ ]:
# Salvando novamente os dados manipulados
# Esse conjunto de dados será utilizado para o dashboard no Power BI.

df.to_csv('divvy_tripdata_final_ckpoint2.csv', index_label = False)

# 5. Perguntas de Negócio

## 5.1 Como os clientes membros e casuais utilizam as bicicletas?

Inicialmente, iremos separar o dataset entre clientes membros e casuais. Após isso, realizaremos uma análise semelhante à feita anteriormente, para cada subset. 

In [ ]:
# Criando checkpoint 2.
df = pd.read_csv('divvy_tripdata_final_ckpoint2.csv')

In [ ]:
# Membros
df_membro = df[df['Tipo Plano'] == 'member']
df_membro.reset_index(inplace = True, drop = True)

# Casual
df_casual = df[df['Tipo Plano'] == 'casual']
df_casual.reset_index(inplace = True, drop = True)

In [ ]:
print('Estatísticas do tempo de aluguél das bicicletas (minutos) - Membros\n')
print(round(df_membro['Duração_Minutos'].describe(), 2))
print(50 * '-')
print('Estatísticas do tempo de aluguél das bicicletas (minutos) - Casuais\n')
print(round(df_casual['Duração_Minutos'].describe(), 2))

In [ ]:
plt.figure(figsize = (15, 6))

plt.subplot(1, 2, 1)
sns.boxplot(df_membro['Duração_Minutos'], color = '#ff9f84'); 
plt.title( "Duração - Clientes Membros" , size = 24 )

plt.subplot(1, 2, 2)
sns.boxplot(df_casual['Duração_Minutos'], color = '#89ffa0'); 
plt.title( "Duração - Clientes Casuais" , size = 24 )


plt.tight_layout()

In [ ]:
plt.figure(figsize = (15, 6))

plt.subplot(1, 2, 1)
df_membro['Duração_Minutos'].hist(color = '#ff9f84', alpha = 0.7, edgecolor='black');
plt.title( "Duração - Clientes Membros" , size = 24 )

plt.subplot(1, 2, 2)
df_casual['Duração_Minutos'].hist(color = '#89ffa0', alpha = 0.7, edgecolor='black');
plt.title( "Duração - Clientes Casuais" , size = 24 )


plt.tight_layout()

In [ ]:
ordem = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira', 'sábado', 'domingo']

plt.figure(figsize = (15, 6))

plt.subplot(1, 2, 1)
sns.countplot(data = df_membro,
                     x = 'Dia da Semana',
                     color = '#ff9f84',
                     alpha = 0.9,
                     order = ordem)

plt.title( "Aluguéis de Bicicleta por Dia da Semana - Clientes Membros" , size = 15 )
plt.xlabel(' ')
plt.ylabel(' ')
plt.xticks(rotation = 45)

plt.subplot(1, 2, 2)
sns.countplot(data = df_casual,
                     x = 'Dia da Semana',
                     color = '#89ffa0',
                     alpha = 0.9,
                     order = ordem)

plt.title( "Aluguéis de Bicicleta por Dia da Semana - Clientes Casuais" , size = 15 )
plt.xlabel(' ')
plt.ylabel(' ')
plt.xticks(rotation = 45)

plt.tight_layout()

> Das análises acima, podemos perceber:

* os membros, em média utilizam as bicicletas menos tempo do que os clientes casuais.
* Os clientes casuais utilizam as bicicletas mais durante os finais de semana, enquanto os membros usam no meio da semana. Porém, existem diversos clientes casuais que também utilizam as bicicletas no meio da semana.

### Clientes com Trabalho igual a Sim

As análises, a partir de agora serão focadas naqueles registros que se enquadram na categoria trabalho, criada anteriormente. Para relembrar, aqueles registros que possuem duração abaixo de 50 minutos e que não seja no final de semana, tem valor `Sim` para essa categoria, indicando que aquele aluguél, **possivelmente**, representa um cliente no deslocamento ao trabalho.

In [ ]:
print('Porcentagem de Bicicletas para o Trabalho - Membros\n')
print(round((df_membro['Trabalho'].value_counts()/df_membro.shape[0])*100, 2))
print(50 * '-')
print('Porcentagem de Bicicletas para o Trabalho - Casuais\n')
print(round((df_casual['Trabalho'].value_counts()/df_casual.shape[0])*100, 2))

> Da informação acima, podemos concluir que muitos registros de clientes casuais (52%, aproximadamente), se enquadram nas características para o deslocamento de trabalho. Em um contexto geral, podemos dizer que 52% dos registros casuais se comportam de maneira semelhante à 73% de registros de clientes membros, indicando um padrão nos dados.

In [ ]:
# Subset dos membros
df_membro_trabalho = df[(df['Tipo Plano'] == 'member') & (df['Trabalho'] == 'Sim')]
df_membro_trabalho.reset_index(inplace = True, drop = True)

# Subset dos casuais
df_casual_trabalho = df[(df['Tipo Plano'] == 'casual') & (df['Trabalho'] == 'Sim')]
df_casual_trabalho.reset_index(inplace = True, drop = True)

In [ ]:
plt.figure(figsize = (15, 6))

plt.subplot(1, 2, 1)
df_membro_trabalho['Duração_Minutos'].hist(color = '#ff9f84', alpha = 0.7, edgecolor='black');
plt.title( "Duração - Clientes Membros (Trabalho)" , size = 24 )

plt.subplot(1, 2, 2)
df_casual_trabalho['Duração_Minutos'].hist(color = '#89ffa0', alpha = 0.7, edgecolor='black');
plt.title( "Duração - Clientes Casuais (Trabalho)" , size = 24 )


plt.tight_layout()

In [ ]:
ordem = ['segunda-feira', 'terça-feira', 'quarta-feira', 'quinta-feira', 'sexta-feira']

plt.figure(figsize = (15, 6))

plt.subplot(1, 2, 1)
sns.countplot(data = df_membro_trabalho,
                     x = 'Dia da Semana',
                     color = '#ff9f84',
                     alpha = 0.9,
                     order = ordem)

plt.title( "Aluguéis por Dia da Semana - Clientes Membros  (Trabalho)" , size = 15 )
plt.xlabel(' ')
plt.ylabel(' ')
plt.xticks(rotation = 45)

plt.subplot(1, 2, 2)
sns.countplot(data = df_casual_trabalho,
                     x = 'Dia da Semana',
                     color = '#89ffa0',
                     alpha = 0.9,
                     order = ordem)

plt.title( "Aluguéis por Dia da Semana - Clientes Casuais (Trabalho)" , size = 15 )
plt.xlabel(' ')
plt.ylabel(' ')
plt.xticks(rotation = 45)

plt.tight_layout()

### Conclusão

* Como os clientes membros e casuais utilizam as bicicletas?

Os clientes membros, em sua maioria, utilizam as bicicletas de segunda à sexta, com duração do aluguél de até 50 minutos. Para a nossa análise, registros que se enquadrassem nessas condições seriam considerados aluguéis de bicicleta para o deslocamnento ao trabalho. 

Já, para os clientes casuais, os registros são equilibrados (50/50 aproximadamente), entre aluguéis que se enquadrem nas condições acima e aluguéis que não se enquadram. Uma suposição feita é que os clientes casuais utilizam as bicicletas para lazer mais do que os clientes membros. 

Ambos os tipos de clientes utilizam mais as bicicletas durante o verão. 

Portanto, podemos concluir que, existem diversos clientes casuais que se comportam como a maior parte dos clientes membros. Considerando as suposições feitas, existem muitos clientes casuais que utilizam as bicicletas para se deslocarem ao trabalho.

## 5.2 Por que os passageiros casuais iriam querer adquirir planos anuais?

Não sabemos as caracteristicas de cada tipo de plano. Porém, devido à alta quantidade de registros de clientes membros que utilizm as bicicletas para o deslocamento ao trabalho, imagina- se que existam benefícios em sua assinatura. Portanto, para os clientes casuais que utilizam as bicicletas para o deslocamento ao trabalho, o plano anual de membro seria a melhor opção, principalmente para aqueles que utilizem com maior frequência.

## 5.3 Qual a estratégia de marketing poderá ser adotada para influenciar os passageiros casuais a se tornarem membros?

1. As campanhas de marketing devem ser voltadas, principalmente, para clientes casuais que utilizem as bicicletas no deslocamento ao trabalho. 

2. Os benefícios que esses clientes receberão com a mudança do plano devem ser reforçados. Algum tipo de promoção poderia ser feita para atrair a atenção desses clientes casuais, além de outros possíveis clientes. 

3. O período ideal para a aplicação da estratégia é no início do verão, quando se tem uma maior utilização das bicicletas.

> Caso haja interesse em realizar alguma estratégia de marketing (ou até algum tipo de pesquisa) in loco, as 5 estações com maiores movimentações são:

* Streeter Dr & Grand Ave

* Michigan Ave & Oak St

* DuSable Lake Shore Dr & North Blvd

* Millennium Park

* Wells St & Concord Ln 


# 6. Próximos Passos

Essa análise considerou dados dos anos 2021 e 2022, anos que ainda sofreram com a influência da pandemia de COVID-19. Por isso, é interessante, futuramente, realizar uma nova análise, considerando um período com menos influência da pandemia e entender se existe alguma mudança.

Além disso, 2 pontos poderiam ser adicionados no conjunto de dados:

**1. Dados financeiros dos tipos de plano**
> Com isso, é possível realizar também uma análise financeira, demonstrando o retorno monetário que as ações sugeridas poderiam trazer.

**2. Dados de clientes**
> Tendo apenas uma coluna de ID do Cliente (mantendo, assim, o anonimato), seria possível realizar análises a nivel de clientes. Isso nos retornaria a quantidade de clientes membros ou casuais, quais clientes utilizam as bicicletas de modo contínuo (vários dias da semana) e quais utilizam de forma esporádica, além de prever a quantidade de clientes que poderiam mudar de plano com a aplicação das ações sugeridas.